In [6]:
import random
import numpy as np
import math

In [335]:
# 1 Unreliable Transmission

def unreliable_transmission(in_frame, p):
    
    
    dt_bits = len(in_frame) # wordsize = 10
    err_bits = int(dt_bits * p) # find num of err bit = 10*0.2 = 2
    
    out_frame = in_frame 
    
    err_index = np.zeros((dt_bits), dtype=int) # [0,0,..] เอาไว้ mark ว่า position ไหนบ้างที่จะ err
    
    x = 0
    while x < err_bits: # 2 loop 
        randindex = random.randint(0, (dt_bits - 1)) # find randomly position of err bit rand(0, 9) = 0 # preventing to err 
        if (err_index[randindex] == 0): # err[0] = 0 -> 1 mark 
            err_index[randindex] = 1
            x += 1
    
    i = 0
    while i < dt_bits: # size 10
        # print("index[{}]: {} ".format(i, out_frame[i]))
        if (err_index[i] == 1):  
            # print(str((int(out_frame[i])+1)%2)) 
            out_frame = out_frame[:i] + str((int(out_frame[i])+1)%2) + out_frame[i+1:] # 0 => (0+1)%2 = 1 | 1 => (1+1)%2 = 0
        i += 1
        
    print("p = {} and dataword size = {} ".format(p, len(in_frame)))
    print("number of corrupted frame: {}*{} = {}".format(p, len(in_frame), err_bits))
    print("----------------------------------------------------------------------------")
    print("sent: ", in_frame)
    print("received: {} with {} error".format(out_frame, err_bits))
            
    # return out_frame
    

In [336]:
unreliable_transmission('1001110010', 0.2)

p = 0.2 and dataword size = 10 
number of corrupted frame: 0.2*10 = 2
----------------------------------------------------------------------------
sent:  1001110010
received: 1011110011 with 2 error


In [7]:
# 2 Parity Bit

# generate one dimession parity
def parity_1D(dtw, pTyp):
    cw = []
    for x in dtw:
        i = 0
        pBit = x[i]
        while i < len(x)-1:
            if (pBit == x[i+1]):
                pBit = '0'
            else:
                pBit = '1'
            i += 1
        if ((pBit == '0' and pTyp == 0) or (pBit == '1' and pTyp == 1)):
            cw.append(x + '0')
        else:
            cw.append(x + '1')
    return cw
# ------------------------------------------------------------------------------------------------------------

# fulfil some data which have size < word_size by '0' (e.g. word_size: 5, dataword: '110', output: '00110')
def fulfilFrame(dtw, w_size):
    # dtw = dataword input as the array
    output = []
    for x in dtw:
        if (len(x) < w_size):
            zero = '0'
            x =  x.zfill(w_size)
        output.append(x)
    return output
# ------------------------------------------------------------------------------------------------------------

# parity generator
def parity_gen(dataword, word_size, parity_type, arr_size):
    
    codeword = []
    cw_col = ''
    
    if (parity_type == '1D-even'):
        codeword = parity_1D(dataword, 0)
        
    elif (parity_type == '1D-odd'):
        codeword = parity_1D(dataword, 1)
        
    elif (parity_type == '2D-even'):
        codeword = fulfilFrame(dataword, word_size)
        codeword = parity_1D(codeword, 0)
        j = 0
        while j < (word_size+1):
            d = codeword[0]
            pBit = d[j]
            k = 0
            while k < (arr_size-1):
                dtw = codeword[k+1]
                if (pBit == dtw[j]):
                    pBit = '0'
                else:
                    pBit = '1'
                k += 1
                
            if (pBit == '0'):
                cw_col += '0' #even (adding '0')
            else:
                cw_col += '1' #odd (adding '1')
                
            j += 1
        codeword.append(cw_col)
    
    elif (parity_type == '2D-odd'):
        codeword = fulfilFrame(dataword, word_size)
        codeword = parity_1D(codeword, 1)
        j = 0
        while j < (word_size+1):
            d = codeword[0]
            pBit = d[j]
            k = 0
            while k < (arr_size-1):
                dtw = codeword[k+1]
                if (pBit == dtw[j]):
                    pBit = '0'
                else:
                    pBit = '1'
                k += 1
                
            if (pBit == '0'):
                cw_col += '1' #even (adding '1')
            else:
                cw_col += '0' #odd (adding '0')
            j += 1
        codeword.append(cw_col)
          
    else:
        print('partity is undefined')
        
    return codeword
# ------------------------------------------------------------------------------------------------------------

# Parity Check
def parity_check(codeword, parity_type, arr_size):
    validity = 1 # Pass = 1, Fail = 0
    pBit = ''
    word_size = len(codeword[0])
    
    # check each row
    for x in codeword:
        i = 0
        pBit = x[i]
        while i < len(x)-1:
            if (pBit == x[i+1]):
                pBit = '0'
            else:
                pBit = '1'
            i += 1
        if ((pBit == '0' and (parity_type == '1D-odd' or parity_type == '2D-odd')) or (pBit == '1' and (parity_type == '1D-even' or parity_type == '2D-even'))):
            return 0

    # check each column        
    if (parity_type == '2D-even' or parity_type == '2D-odd'):
        j = 0
        while j < word_size:
            c = codeword[0]
            pBit = c[j]
            k = 0
            while k < (arr_size-1):
                cw = codeword[k+1]
                if (pBit == cw[j]):
                    pBit = '0'
                else:
                    pBit = '1'
                k += 1
            if ((pBit == '0' and parity_type == '2D-odd') or (pBit == '1' and parity_type == '2D-even')):
                return 0
            j += 1
    
    return validity
# ------------------------------------------------------------------------------------------------------------

In [8]:
dtw = np.array(['10110', '0011', '1010'])
parity_gen(dtw, 5, '1D-even', 3)

['101101', '00110', '10100']

In [12]:
dtw = np.array(['1010101', '0100110', '0101010'])
parity_gen(dtw, 7, '2D-even', 3)

['10101010', '01001101', '01010101', '10110010']

In [9]:
dtw = np.array(['1010111'])
parity_gen(dtw, 7, '1D-even', 3)

['10101111']

In [339]:
parity_gen(dtw, 5, '1D-odd', 3)

['101100', '00111', '10101']

In [340]:
dtw_2d = np.array(['10111', '10101', '1110'])
parity_gen(dtw_2d, 5, '2D-even', 3)

['101110', '101011', '011101', '011000']

In [341]:
parity_gen(dtw_2d, 5, '2D-odd', 3)

['101111', '101010', '011100', '100110']

In [342]:
cw = ['101111', '101010', '011100', '100110'] # Pass: 1
parity_check(cw, '1D-odd', 4)

1

In [343]:
parity_check(cw, '1D-even', 4) # Fail: 0

0

In [344]:
parity_check(cw, '1D-even', 4)

0

In [69]:
d = np.array(['1101001', '1001000', '1110110'])
parity_gen(d, 7, '1D-even', 3)

['11010010', '10010000', '11101101']

In [18]:
# 3 CRC

# xor of 2 binary numbers
def XOR(d, s):
    i = 0
    rmd = ''
    while i < len(d):
        if (d[i] == s[i]):
            rmd = rmd + '0'
        else:
            rmd = rmd + '1'
        i += 1 
    return rmd[1:] # e.g.rmd = '0110', only need '110'
# ------------------------------------------------------------------------------------------------------------

# modulo-2 Division
def mod2div(dividend, divisor): # '101101100000'/'11111'  => 10110/11111 = 01001 (xor)
    p = len(divisor)            # ' 10011'/'11111' => 01100
    
    # zero divisor # 01110 / 00000 = 10001 
    zerodiv = '0'
    zerodiv = zerodiv.zfill(p) 
    
    sel_dividend = dividend[:(p)] # e.g. '110100'/'1001' => first sel_dividend = '1101'
    
    while p <= len(dividend):
        if (sel_dividend[0] == '1'):
            # print("p{}: {} xor {} = {}".format(p, sel_dividend, divisor, XOR(sel_dividend, divisor)))
            sel_dividend = XOR(sel_dividend, divisor)
            if (p != len(dividend)): 
                sel_dividend = sel_dividend + dividend[p]
        else:
            # print("p{}: {} xor {} = {}".format(p, sel_divdend, zerodiv, XOR(sel_divdend, zerodiv)))
            sel_dividend = XOR(sel_dividend, zerodiv) 
            if (p != len(dividend)): 
                sel_dividend = sel_dividend + dividend[p]    
        p += 1
    
    return sel_dividend
# ------------------------------------------------------------------------------------------------------------

# CRC Generator
def CRC_gen(dataword, word_size, CRC_type):
    
    divisors = ['11111', '111010101', '10100000000000011', '11000000000000101', '1100000000101000100000001', '100000100110000010001110110110111']
    dvs = '10101'
    
    extraBits = '0' 
    extraBits = extraBits.zfill(len(dvs)-1) # input dataword '10110110' => divisor '11111' dividend => '101101100000'
    
    dtw = dataword + extraBits
    # print(dtw)
    
    remainder = mod2div(dtw, dvs)
    print(remainder)
        
    return dataword+remainder
# ------------------------------------------------------------------------------------------------------------

# CRC Checker
def CRC_check(codeword, CRC_type):
    
    # divisors = [CRC-4, CRC-8, Reversed CRC-16, CRC-16, CRC-24, CRC-32]
    divisors = ['11111', '111010101', '10100000000000011', '11000000000000101', '1100000000101000100000001', '100000100110000010001110110110111']
    dvs = divisors[CRC_type]
    
    remainder = mod2div(codeword, dvs)
    # print(remainder) int('000000', 2) = 0 cast binary to integer
    if (int(remainder, 2) == 0):
        return 1 # Pass (no error)
    else:
        return 0 # Fail (error occur)
# ------------------------------------------------------------------------------------------------------------

In [19]:
CRC_gen('10101011', 8, 0)

1111


'101010111111'

In [349]:
CRC_check('1001001101', 0)

1

In [1]:
# 4 Checksum

# sum of 2 binary numbers
def sumBin(bin1, bin2):
    out =  bin(int(bin1, 2) + int(bin2, 2))
    return out[2:]   
# ------------------------------------------------------------------------------------------------------------

# find one's complement of an input binary number
def complement(bin):
    out = bin
    i = 0  
    while i < len(bin):
        # print("{} {} => {} {}".format(out, out[i], out[:i], out[i+1:]))
        out = out[:i] + str((int(out[i])+1)%2) + out[i+1:]
        i += 1
    return out
# ------------------------------------------------------------------------------------------------------------

# Checksum Generator
def Checksum_gen(dataword, word_size, num_blocks): #3 block dataword 
    i = 0
    sum_ = dataword[0]
    codeword = dataword
    while i < num_blocks-1:
        sum_ = sumBin(sum_, dataword[i+1])
        if (len(sum_) > word_size):
            #print("{} => {} + {} ".format(sum_, sum_[len(sum_)-word_size:word_size+1], sum_[:len(sum_)-word_size]))
            sum_ = sumBin(sum_[:len(sum_)-word_size], sum_[len(sum_)-word_size:word_size+1])
        i += 1
    if (len(sum_) < word_size): 
        for i in range((word_size-len(sum_))):
                sum_ = '0' + sum_
                
    codeword.append(complement(sum_))
    return codeword
# ------------------------------------------------------------------------------------------------------------        

# Checksum Checker
def Checksum_check(codeword, word_size, num_blocks): # 4 block = 0000 no err 
    i = 0
    sum_cw = codeword[0]
    while i < num_blocks:
        sum_cw = sumBin(sum_cw, codeword[i+1])
        if (len(sum_cw) > word_size):
            sum_cw = sumBin(sum_cw[:len(sum_cw)-word_size], sum_cw[len(sum_cw)-word_size:word_size+1])
        i += 1
        
    if (len(sum_cw) < word_size): 
        for i in range((word_size-len(sum_cw))):
                sum_cw = '0' + sum_cw
                
    if (int(complement(sum_cw), 2) == 0):
        return 1
    else:
        return 0
# ------------------------------------------------------------------------------------------------------------

In [2]:
dtw = ['1011', '0011', '1010']
Checksum_gen(dtw, 4, 3)

['1011', '0011', '1010', '0110']

In [351]:
dtw = ['10011001', '11100010', '00100100', '10000100']
Checksum_gen(dtw, 8, 4)

['10011001', '11100010', '00100100', '10000100', '11011010']

In [352]:
dtw = ['01111', '10111', '11000', '01101']
Checksum_gen(dtw, 5, 4)

['01111', '10111', '11000', '01101', '10010']

In [353]:
cw = ['01111', '10111', '11000', '01101', '10010'] # no error bit
cw_err = ['11111', '10111', '11000', '01101', '10010'] # 1 error bit
Checksum_check(cw, 5, 4)

1

In [354]:
Checksum_gen(['00110111', '10101010'],  8, 2)

['00110111', '10101010', '00011110']

In [10]:
# 5 Hamming Code

# return xor of 2 bits
def xor(bit1, bit2):
    if (bit1 == bit2):
        return '0'
    else:
        return '1'
# ------------------------------------------------------------------------------------------------------------        

# Hammin Code Generator
def Hamming_gen(dataword):
    
    codeword = dataword[::-1]
    
    # find number of parity bits
    numPBits = math.ceil(math.log(len(dataword), 2))
    if (len(dataword) > ((2**numPBits)-numPBits-1)): # 2^m - m - 1
        numPBits += 1
        
    x = 0  
    k = 0
    # add parity bits into codeword
    while x < numPBits:
        # print("{} {} => {} {}".format(out, out[i], out[:i], out[i+1:]))
        codeword = codeword[:k] + '0' + codeword[k:]
        k += pow(2, x) 
        x += 1
    
    # find each parity bit
    i = 0
    seq = np.ones(numPBits, dtype=int)*(-1)
    while i < len(codeword):
        j = 0
        # print("index {} ({})".format(i+1, i))
        while j < numPBits:
            if ((i+1)%(pow(2, j)) == 0 and seq[j] == -1): # first digit that cover
                #print("  PBit {}: ".format(pow(2, j)))
                seq[j] = 1
                currentBit = codeword[i]
                codeword = codeword[:pow(2, j)-1] + xor(codeword[pow(2, j)-1], currentBit) + codeword[pow(2, j):]

            elif (seq[j] < pow(2, j) and seq[j] > -1): # every parity bit (e.g.p2 => 2 bit will be calculated by xor)
                #print("  PBit {}: ".format(pow(2, j)))
                seq[j] += 1
                currentBit = codeword[i]
                #print("  {} xor {} = {}".format(currentBit, codeword[pow(2, j)-1], xor(codeword[pow(2, j)-1], currentBit)))
                codeword = codeword[:pow(2, j)-1] + xor(codeword[pow(2, j)-1], currentBit) + codeword[pow(2, j):]
                    
            elif (seq[j] >= pow(2, j) and seq[j] < 2*pow(2, j)-1):
                seq[j] += 1
                    
            elif (seq[j] == 2*pow(2, j)-1):
                seq[j] = 0
            j += 1
        # print("--------------------------------------------")
        i += 1
            
    return codeword[::-1]
# ------------------------------------------------------------------------------------------------------------

# Hamming Code Checker
def Hamming_check(in_codeword):
    err_position = ''
    
    codeword = in_codeword[::-1]
    
    # find number of parity bits
    numPBits = math.ceil(math.log(len(in_codeword), 2))
    if (len(in_codeword) > ((2**numPBits)-1)): # 2^m - 1
        numPBits += 1
    
    i = 0
    bit = np.zeros(numPBits, dtype=int)
    while i < numPBits:
        j = pow(2, i) - 1 
        count = 0
        while j < len(codeword):
            if (count == 0 or count < pow(2, i)): # (0-3)
                #print("  {} xor {} = {}".format(bit[i], codeword[j], xor(str(bit[i]), codeword[j])))
                bit[i] = xor(str(bit[i]), codeword[j])
                #print(bit[i])
                count += 1
                j += 1
            else:
                count = 0
                j += pow(2, i)
        err_position = str(bit[i]) + err_position
        i += 1
    
    if (int(err_position, 2) == 0):
        return -1 # remainder is not non-zeros = no error
    else:
        print("{} '{}' {}".format(in_codeword[:len(in_codeword)-int(err_position, 2)], in_codeword[int(err_position, 2)-1], in_codeword[len(in_codeword)-int(err_position, 2)+1:]))
        print("location of error is ", int(err_position, 2))
    
    # return int(err_position, 2)
# ------------------------------------------------------------------------------------------------------------

In [356]:
Hamming_check('10011100101') # no error

-1

In [11]:
Hamming_gen('1110001')

'11110000111'

In [358]:
Hamming_gen('10011100101')

'100111000101111'

In [4]:
Hamming_check('100111000100111')

10011100010 '1' 111
location of error is  4
